# Prerequisites


In [58]:
!pip install glob2

In [59]:
!pip install pytest-shutil

In [60]:
!pip install SimpleITK

In [61]:
from glob import glob
import shutil
import os

import SimpleITK as sitk
import time
import re


# Convertion of .nii files to .dcm files without the help of slicer

In [62]:
def writeSlices(series_tag_values, new_img, i, out_dir):
    image_slice = new_img[:,:,i]
    
    # Set the pixel type explicitly to int16.
    image_slice = sitk.Cast(image_slice, sitk.sitkInt16)
    
    
    writer = sitk.ImageFileWriter()
    writer.KeepOriginalImageUIDOn()

    # Tags shared by the series.
    list(map(lambda tag_value: image_slice.SetMetaData(tag_value[0], tag_value[1]), series_tag_values))

    # Slice specific tags.
    image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d")) # Instance Creation Date
    image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S")) # Instance Creation Time

    # Setting the type to CT preserves the slice location.
    image_slice.SetMetaData("0008|0060", "CT")  # set the type to CT so the thickness is carried over

    # (0020, 0032) image position patient determines the 3D spacing between slices.
    image_slice.SetMetaData("0020|0032", '\\'.join(map(str,new_img.TransformIndexToPhysicalPoint((0,0,i))))) # Image Position (Patient)
    image_slice.SetMetaData("0020,0013", str(i)) # Instance Number

    # Write to the output directory and add the extension dcm, to force writing in DICOM format.
    writer.SetFileName(os.path.join(out_dir,'slice' + str(i).zfill(4) + '.dcm'))
    writer.Execute(image_slice)

In [63]:
def nifti2dicom_1file(in_dir, out_dir):
    """
    This function is to convert only one nifti file into dicom series
    `nifti_dir`: the path to the one nifti file
    `out_dir`: the path to output
    """

    os.makedirs(out_dir, exist_ok=True)

    new_img = sitk.ReadImage(in_dir) 
    modification_time = time.strftime("%H%M%S")
    modification_date = time.strftime("%Y%m%d")

    direction = new_img.GetDirection()
    series_tag_values = [("0008|0031",modification_time), # Series Time
                    ("0008|0021",modification_date), # Series Date
                    ("0008|0008","DERIVED\\SECONDARY"), # Image Type
                    ("0020|000e", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time), # Series Instance UID
                    ("0020|0037", '\\'.join(map(str, (direction[0], direction[3], direction[6],# Image Orientation (Patient)
                                                        direction[1],direction[4],direction[7])))),
                    ("0008|103e", "Created-Pycad")] # Series Description

    # Write slices to output directory
    list(map(lambda i: writeSlices(series_tag_values, new_img, i, out_dir), range(new_img.GetDepth())))

In [64]:
def nifti2dicom_mfiles(nifti_dir, out_dir=''):
    """
    This function is to convert multiple nifti files into dicom files
    `nifti_dir`: You enter the global path to all of the nifti files here.
    `out_dir`: Put the path to where you want to save all the dicoms here.
    PS: Each nifti file's folders will be created automatically, so you do not need to create an empty folder for each patient.
    """

    images = glob(nifti_dir + '/*.nii.gz')

    for image in images:
        o_path = out_dir + '/' + os.path.basename(image)[:-7]
        os.makedirs(o_path, exist_ok=True)

        nifti2dicom_1file(image, o_path)

# Making relevant folder

## give the input directory path

In [65]:
in_dir = "E:/Thesis/dataset-work"


In [66]:
root = in_dir
print(root)
new_folder_1 = "post-ses"
new_folder_2 = "pre-ses"
new_folder_path_1 = os.path.join(root, new_folder_1)
new_folder_path_2 = os.path.join(root, new_folder_2)

try:
    os.mkdir(new_folder_path_1)
    os.mkdir(new_folder_path_2)
except FileExistsError:
    print(f"Folder {new_folder_path_1} & {new_folder_path_2} already exists")
else:
    print(f"Folder {new_folder_path_1} & {new_folder_path_2} created successfully")


anat_folder = "anat"
func_folder = "func"
anat_folder_path_1 = os.path.join(new_folder_path_1, anat_folder)
anat_folder_path_2 = os.path.join(new_folder_path_2, anat_folder)

func_folder_path_1 = os.path.join(new_folder_path_1, func_folder)
func_folder_path_2 = os.path.join(new_folder_path_2, func_folder)

## making anat and func folder in both post-ses and pre-ses
try:
    os.mkdir(anat_folder_path_1)
    os.mkdir(func_folder_path_1)
except FileExistsError:
    print(f"Folder {anat_folder_path_1} & {func_folder_path_1} already exists")
else:
    print(f"Folder {anat_folder_path_1} created successfully")
    print(f"Folder {func_folder_path_1} created successfully")


try:
    os.mkdir(anat_folder_path_2)
    os.mkdir(func_folder_path_2)
except FileExistsError:
    print(f"Folder {anat_folder_path_2} & {func_folder_path_2} already exists")
else:
    print(f"Folder {anat_folder_path_2} created successfully")
    print(f"Folder {func_folder_path_2} created successfully")

    

E:/Thesis/dataset-work
Folder E:/Thesis/dataset-work\post-ses & E:/Thesis/dataset-work\pre-ses already exists
Folder E:/Thesis/dataset-work\post-ses\anat & E:/Thesis/dataset-work\post-ses\func already exists
Folder E:/Thesis/dataset-work\pre-ses\anat & E:/Thesis/dataset-work\pre-ses\func already exists


In [67]:
# Base directory where the 10 folders are located
base_directory = in_dir

# List all the directories in the base directory
all_folders = os.listdir(base_directory)

# Filter and create a list of folders with the "sub-" prefix
sub_folders = [folder for folder in all_folders if folder.startswith("sub-")]

print(sub_folders)

['sub-00', 'sub-01', 'sub-02', 'sub-03', 'sub-05', 'sub-07', 'sub-10']


In [68]:
from pathlib import Path

# List of folder names
folder_names = sub_folders

# Loop through each folder
for folder_name in folder_names:
    pre_ses_anat = os.path.join(base_directory, folder_name, 'ses-pre', 'anat')
    pre_ses_func = os.path.join(base_directory, folder_name, 'ses-pre', 'func')
    
    post_ses_anat = os.path.join(base_directory, folder_name, 'ses-post', 'anat')
    post_ses_func = os.path.join(base_directory, folder_name, 'ses-post', 'func')
    
    base_pre_ses_anat = os.path.join(base_directory, 'pre-ses', 'anat')
    base_pre_ses_func = os.path.join(base_directory, 'pre-ses', 'func')
    
    base_post_ses_anat = os.path.join(base_directory, 'post-ses', 'anat')
    base_post_ses_func = os.path.join(base_directory, 'post-ses', 'func')
    
    print(folder_name)
    
    
    # for filename in os.listdir(pre_ses_anat):
    #     f = os.path.join(pre_ses_anat, filename)
    #     # checking if it is a file
    #     if os.path.isfile(f):
    #         print(f)
    
    for file in os.listdir(pre_ses_anat):
        source_file_path = os.path.join(pre_ses_anat, file)
        destination_file_path = os.path.join(base_pre_ses_anat, file)

        # Check if the current item is a file (not a directory)
        if os.path.isfile(source_file_path):
            shutil.copy2(source_file_path, destination_file_path)
            print(f"File '{file}' copied to '{base_pre_ses_anat}'")   
        else:
            print("error not copied")
            
    for file in os.listdir(pre_ses_func):
        source_file_path = os.path.join(pre_ses_func, file)
        destination_file_path = os.path.join(base_pre_ses_func, file)

        # Check if the current item is a file (not a directory)
        if os.path.isfile(source_file_path):
            shutil.copy2(source_file_path, destination_file_path)
       
            
            
    for file in os.listdir(post_ses_anat):
        source_file_path = os.path.join(post_ses_anat, file)
        destination_file_path = os.path.join(base_post_ses_anat, file)

        # Check if the current item is a file (not a directory)
        if os.path.isfile(source_file_path):
            shutil.copy2(source_file_path, destination_file_path)
         
            
    for file in os.listdir(post_ses_func):
        source_file_path = os.path.join(post_ses_func, file)
        destination_file_path = os.path.join(base_post_ses_func, file)

        # Check if the current item is a file (not a directory)
        if os.path.isfile(source_file_path):
            shutil.copy2(source_file_path, destination_file_path)
        

sub-00
File 'sub-00_ses-pre_T1w.json' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
File 'sub-00_ses-pre_T1w.nii.gz' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
sub-01
File 'sub-01_ses-pre_T1w.json' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
File 'sub-01_ses-pre_T1w.nii.gz' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
File 'sub-01_ses-pre_T1w_label-lesion_roi.nii.gz' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
sub-02
File 'sub-02_ses-pre_T1w.json' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
File 'sub-02_ses-pre_T1w.nii.gz' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
sub-03
File 'sub-03_ses-pre_T1w.json' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
File 'sub-03_ses-pre_T1w.nii.gz' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
sub-05
File 'sub-05_ses-pre_T1w.json' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
File 'sub-05_ses-pre_T1w.nii.gz' copied to 'E:/Thesis/dataset-work\pre-ses\anat'
File 'sub-05_ses-pre_T1w_label-lesion_roi.nii.gz' copied to 'E:/The

In [69]:
def dicom_files(path):
    root = os.path.abspath(path)
    print(root)
    new_folder_1 = "dicom_initial"
    new_folder_2 = "dicom_final"
    new_folder_path_1 = os.path.join(root, new_folder_1)
    new_folder_path_2 = os.path.join(root, new_folder_2)

    try:
        os.mkdir(new_folder_path_1)
        os.mkdir(new_folder_path_2)
    except FileExistsError:
        print(f"Folder {new_folder_path_1} & {new_folder_path_2} already exists")
    else:
        print(f"Folder {new_folder_path_1} & {new_folder_path_2} created successfully")


    labels_folder = "antonmy"
    images_folder = "functional"
    labels_folder_path_1 = os.path.join(new_folder_path_1, labels_folder)
    labels_folder_path_2 = os.path.join(new_folder_path_2, labels_folder)

    images_folder_path_1 = os.path.join(new_folder_path_1, images_folder)
    images_folder_path_2 = os.path.join(new_folder_path_2, images_folder)

    ## making dicom_initial's labels and images
    try:
        os.mkdir(labels_folder_path_1)
        os.mkdir(images_folder_path_1)
    except FileExistsError:
        print(f"Folder {labels_folder_path_1} & {images_folder_path_1} already exists")
    else:
        print(f"Folder {labels_folder_path_1} created successfully")
        print(f"Folder {images_folder_path_1} created successfully")


    ## making dicom_final's labels and images
    try:
        os.mkdir(labels_folder_path_2)
        os.mkdir(images_folder_path_2)
    except FileExistsError:
        print(f"Folder {labels_folder_path_2} & {images_folder_path_2} already exists")
    else:
        print(f"Folder {labels_folder_path_2} created successfully")
        print(f"Folder {images_folder_path_2} created successfully")

In [70]:
post_file_dir = "E:\Thesis\dataset-work\post-ses"
pre_file_dir = "E:\Thesis\dataset-work\pre-ses"


dicom_files(post_file_dir)
dicom_files(pre_file_dir)

E:\Thesis\dataset-work\post-ses
Folder E:\Thesis\dataset-work\post-ses\dicom_initial & E:\Thesis\dataset-work\post-ses\dicom_final already exists
Folder E:\Thesis\dataset-work\post-ses\dicom_initial\antonmy & E:\Thesis\dataset-work\post-ses\dicom_initial\functional already exists
Folder E:\Thesis\dataset-work\post-ses\dicom_final\antonmy & E:\Thesis\dataset-work\post-ses\dicom_final\functional already exists
E:\Thesis\dataset-work\pre-ses
Folder E:\Thesis\dataset-work\pre-ses\dicom_initial & E:\Thesis\dataset-work\pre-ses\dicom_final already exists
Folder E:\Thesis\dataset-work\pre-ses\dicom_initial\antonmy & E:\Thesis\dataset-work\pre-ses\dicom_initial\functional already exists
Folder E:\Thesis\dataset-work\pre-ses\dicom_final\antonmy & E:\Thesis\dataset-work\pre-ses\dicom_final\functional already exists


In [71]:
path_1 = "E:/Thesis/dataset-work/post-ses/anat"
path_2 = "E:/Thesis/dataset-work/post-ses/func"

path_1_out = "E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy"
path_2_out = "E:/Thesis/dataset-work/post-ses/dicom_initial/functional"
print(os.listdir(path_1))

['sub-00_ses-post_T1w.json', 'sub-00_ses-post_T1w.nii.gz', 'sub-01_ses-post_T1w.json', 'sub-01_ses-post_T1w.nii.gz', 'sub-02_ses-post_T1w.json', 'sub-02_ses-post_T1w.nii.gz', 'sub-03_ses-post_T1w.json', 'sub-03_ses-post_T1w.nii.gz', 'sub-05_ses-post_T1w.json', 'sub-05_ses-post_T1w.nii.gz', 'sub-07_ses-post_T1w.json', 'sub-07_ses-post_T1w.nii.gz', 'sub-10_ses-post_T1w.json', 'sub-10_ses-post_T1w.nii.gz', 'temp']


In [72]:
import pathlib
# List all files in the source folder

files_1 = os.listdir(path_1)

temp = os.path.join(path_1 , "temp")

try:
    os.mkdir(temp)
except FileExistsError:
    print(f"Folder {temp}")
else:
    print(f"Folder {temp} created successfully")
    

Folder E:/Thesis/dataset-work/post-ses/anat\temp


In [73]:
source_folder = "E:/Thesis/dataset-work/post-ses/anat"
destination_folder = os.path.normpath(temp)
print(destination_folder)
# Iterate through each file and move if it has the desired extension
for file in files_1:
    k = pathlib.Path(file).suffix
    print(k)
    if k == ".gz":
        source_file_path = os.path.join(source_folder, file)
        destination_file_path = os.path.join(destination_folder, file)
        
        # Move the file
        shutil.move(source_file_path, destination_file_path)
        print(f"File '{file}' moved to '{destination_folder}'")

E:\Thesis\dataset-work\post-ses\anat\temp
.json
.gz
File 'sub-00_ses-post_T1w.nii.gz' moved to 'E:\Thesis\dataset-work\post-ses\anat\temp'
.json
.gz
File 'sub-01_ses-post_T1w.nii.gz' moved to 'E:\Thesis\dataset-work\post-ses\anat\temp'
.json
.gz
File 'sub-02_ses-post_T1w.nii.gz' moved to 'E:\Thesis\dataset-work\post-ses\anat\temp'
.json
.gz
File 'sub-03_ses-post_T1w.nii.gz' moved to 'E:\Thesis\dataset-work\post-ses\anat\temp'
.json
.gz
File 'sub-05_ses-post_T1w.nii.gz' moved to 'E:\Thesis\dataset-work\post-ses\anat\temp'
.json
.gz
File 'sub-07_ses-post_T1w.nii.gz' moved to 'E:\Thesis\dataset-work\post-ses\anat\temp'
.json
.gz
File 'sub-10_ses-post_T1w.nii.gz' moved to 'E:\Thesis\dataset-work\post-ses\anat\temp'



### nifti to dicom (stored in dicom_initial)

In [75]:
nifti2dicom_mfiles(temp, path_1_out)

# Divide the .dcm files to multiples folders of same sample size 

In [76]:
print(path_1_out)


E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy


In [77]:
d_initial_labels = path_1_out
d_final_labels = os.path.normpath("E:\Thesis\dataset-work\post-ses\dicom_final\antonmy")
  
print(os.path.normpath(d_initial_labels))
print(d_final_labels)

p = glob( d_initial_labels +"/*")
print(p)
p = p[0]
print(p)
l = len(glob(p + "/*"))
print(l)

E:\Thesis\dataset-work\post-ses\dicom_initial\antonmy
E:\Thesis\dataset-work\post-ses\dicom_finalntonmy
['E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\\sub-00_ses-post_T1w', 'E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\\sub-01_ses-post_T1w', 'E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\\sub-02_ses-post_T1w', 'E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\\sub-03_ses-post_T1w', 'E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\\sub-05_ses-post_T1w', 'E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\\sub-07_ses-post_T1w', 'E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\\sub-10_ses-post_T1w']
E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-00_ses-post_T1w
256


In [78]:
def dicom_divide(number_of_sice, initial_path , final_path):
    for i in glob(initial_path+"/*"):
        print("Entered-> ",i)

        folder_name = os.path.basename(os.path.normpath(i))
        number_of_folder = int(len(glob(i+"/*"))/number_of_slice)
        print(f"folder_name: {folder_name}")
        print(f"numbers_of_folder: {number_of_folder}")


        for j in range(1,number_of_folder+1):
            new_dir = os.path.join(final_path, folder_name+"_"+str(j))
            print(new_dir)
            
            try:
                os.mkdir(new_dir)
            except OSError as e:
                print(f"Error: {new_dir} : {e.strerror}")
                continue
            
            # get files and copy to the new folder 
            k = 0
            for l in glob(i+"/*"):
                if k == number_of_slice:
                    break

                #shutil.copy(l,new_dir) ---- You have to move or the code has to be changed the code don't copy all the files only first 40 when used copy
                shutil.move(l,new_dir)
                k+=1

In [79]:
#number_of_slice = 40

def number_of_slices( folder_path ):
    
    min_samples = float('inf')
    for i in glob(folder_path+"/*"):
        print("Entered-> ",i)
        
        length = len(glob(i + "/*"))
        
        if(length < min_samples):
            min_samples = length
            print(min_samples)
            
    return min_samples   


In [80]:
number_of_slice = number_of_slices(d_initial_labels)
print(number_of_slice)

Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-00_ses-post_T1w
256
Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-01_ses-post_T1w
Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-02_ses-post_T1w
Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-03_ses-post_T1w
Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-05_ses-post_T1w
Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-07_ses-post_T1w
Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-10_ses-post_T1w
256


In [81]:
#for labels
dicom_divide(number_of_slice,d_initial_labels , d_final_labels)

Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-00_ses-post_T1w
folder_name: sub-00_ses-post_T1w
numbers_of_folder: 1
E:\Thesis\dataset-work\post-ses\dicom_finalntonmy\sub-00_ses-post_T1w_1
Error: E:\Thesis\dataset-work\post-ses\dicom_finalntonmy\sub-00_ses-post_T1w_1 : The filename, directory name, or volume label syntax is incorrect
Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-01_ses-post_T1w
folder_name: sub-01_ses-post_T1w
numbers_of_folder: 1
E:\Thesis\dataset-work\post-ses\dicom_finalntonmy\sub-01_ses-post_T1w_1
Error: E:\Thesis\dataset-work\post-ses\dicom_finalntonmy\sub-01_ses-post_T1w_1 : The filename, directory name, or volume label syntax is incorrect
Entered->  E:/Thesis/dataset-work/post-ses/dicom_initial/antonmy\sub-02_ses-post_T1w
folder_name: sub-02_ses-post_T1w
numbers_of_folder: 1
E:\Thesis\dataset-work\post-ses\dicom_finalntonmy\sub-02_ses-post_T1w_1
Error: E:\Thesis\dataset-work\post-ses\dicom_finalntonmy\sub-02_ses

In [82]:
print(len(glob(d_final_labels + "/*")))

0


# Convert to nifti files

In [83]:
!pip install dicom2nifti

In [84]:
import dicom2nifti as dn

### create nifti directory

In [85]:
root = os.path.dirname(in_dir)
print(root)
new_folder_3 = "nifti_initial"
new_folder_path_3 = os.path.join(root, new_folder_3)


try:
    os.mkdir(new_folder_path_3)
except FileExistsError:
    print(f"Folder {new_folder_path_3} already exists")
else:
    print(f"Folder {new_folder_path_3} created successfully")

    
labels_folder = "labels"
images_folder = "images"

labels_folder_path_3 = os.path.join(new_folder_path_3, labels_folder)
images_folder_path_3 = os.path.join(new_folder_path_3, images_folder)


## making nifti_initial's labels and images
try:
    os.mkdir(labels_folder_path_3)
    os.mkdir(images_folder_path_3)
except FileExistsError:
    print(f"Folder {labels_folder_path_3} & {images_folder_path_3} already exists")
else:
    print(f"Folder {labels_folder_path_3} created successfully")
    print(f"Folder {images_folder_path_3} created successfully")

E:/Thesis
Folder E:/Thesis\nifti_initial created successfully
Folder E:/Thesis\nifti_initial\labels created successfully
Folder E:/Thesis\nifti_initial\images created successfully


In [86]:
dicom_path_labels = d_final_labels+ "/*"
nifti_path_labels = labels_folder_path_3

print(dicom_path_labels)
print(nifti_path_labels)


E:\Thesis\dataset-work\post-ses\dicom_finalntonmy/*
E:/Thesis\nifti_initial\labels


In [87]:
def dicom_2_nifti(dicom_path , nifti_path):
    folder_list = glob(dicom_path)
    print(folder_list)
    print(nifti_path)
    counter = 0
    
    for i in folder_list:
        folder_name = os.path.basename(os.path.normpath(i))
        dn.dicom_series_to_nifti(i,os.path.join(nifti_path,folder_name+".nii.gz"))
        print(f"completed_{counter}")
        counter = counter + 1


In [88]:
#for labels
dicom_2_nifti(dicom_path_labels , nifti_path_labels)

[]
E:/Thesis\nifti_initial\labels


# Find NULL

In [26]:
!pip install nibabel

In [27]:
import nibabel as nb
import numpy as np

In [28]:
path_of = nifti_path_labels + "/*"

In [29]:
list_of_null=[]

for i in glob(path_of):
    nifti = nb.load(i)
    
    frame_data = nifti.get_fdata()
    #print(frame_data)
    
    uniq = np.unique(frame_data)
    print(uniq)
    if len(uniq) == 1:
        list_of_null.append(i)

[0. 1.]
[0.]
[0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0.]
[0.]
[0. 1.]
[0.]
[0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]


In [30]:
list_of_null

['E:/Thesis/Most_one\\nifti_initial\\labels\\labels_spleen_12_1.nii.gz',
 'E:/Thesis/Most_one\\nifti_initial\\labels\\labels_spleen_12_2.nii.gz',
 'E:/Thesis/Most_one\\nifti_initial\\labels\\labels_spleen_2_1.nii.gz',
 'E:/Thesis/Most_one\\nifti_initial\\labels\\labels_spleen_2_2.nii.gz',
 'E:/Thesis/Most_one\\nifti_initial\\labels\\labels_spleen_6_1.nii.gz',
 'E:/Thesis/Most_one\\nifti_initial\\labels\\labels_spleen_6_2.nii.gz']

## Remove NULL files

In [31]:
def remove_null(list_of_null):
    for i in list_of_null:
        if os.path.exists(i):
            os.remove(i)
            print("File deleted successfully.")
        else:
            print("File does not exist.")

In [32]:
 #   remove_null(list_of_null)